In [23]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

result = []
# Create a pose landmarker instance with the live stream mode:
def print_result(_result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    global result
    result.append((_result, output_image))

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path="/Users/Ethan/Developer/Projects/College/大四下/人工智慧/final/lotpose_bd/lotpose/checkpoints/pose_landmarker_lite.task"),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)


In [24]:
#@markdown To better demonstrate the Pose Landmarker API, we have created a set of visualization tools that will be used in this colab. These will draw the landmarks on a detect person, as well as the expected connections between those markers.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [26]:
import time
import cv2

with PoseLandmarker.create_from_options(options) as landmarker:
    capture = cv2.VideoCapture('./dance.mp4')

    while True:
        _, frame = capture.read()
        if frame is None:
            break

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        landmarker.detect_async(mp_image, int(time.time() * 1000))

        # annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), pose_landmarker_result)
        # cv2.imshow('MediaPipe Pose', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
        #
        # # Check for 'q' key press to exit
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break


In [30]:
for r, img in result:
    annotated_image = draw_landmarks_on_image(img.numpy_view(), r)
    cv2.imshow('MediaPipe Pose', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

    # Check for 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break